# Extraccion de datos de la API Financial data for every need

In [1]:
#Importacion de librerias
 
import requests
from pprint import pprint
import psycopg2
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
import pandas
from utils_apis import * 
import pandas as pd
from deltalake import DeltaTable
import pyarrow as pa
import os
from dotenv import load_dotenv




In [2]:
base_url = 'https://financialmodelingprep.com/api/v3'


Conectarme a la base de datos donde tengo alojado la fecha de ultima extraccion

In [3]:
# OBTINE LA ULTIMA FECHA DE EXTRACCION
ultimaExtraccion= conexion_consultas(
    'pipeline.conf', 'postgresql','postgresql+psycopg2'
    )
# OBTENER LA SESSION
session= devolver_session( 'pipeline.conf', 'postgresql','postgresql+psycopg2')

print(ultimaExtraccion)
print(session)


2024-11-18 21:06:52.474066+00:00


EXTRACCION FULL

EXTRACCION INCREMENTAL STATEFUL

se extrae de: API de criptomonedas intradía(Proporciona datos de precios intradiarios de todas las criptomonedas que se comercializan en bolsas de todo el mundo.)


In [4]:
load_dotenv(dotenv_path='.env')
api_key = os.getenv("API_KEY")


In [ ]:
""" 
Bloque donde se hace se arma la URL para hacer una peticion GET a la API,
y luego se pasa los parametros a la funcion get_data para convertir los datos obtenidos
en un formato JSON
"""
from datetime import datetime, timezone,timedelta
"""
se convierte la ultima fecha de extraccion a una zona horaria para poder restar con la fecha actual(la actual estaba adelantada)
y lograr traer desde la api solo las fechas posteriores a la ultima extraccion.
"""
ultimaExtraccion = ultimaExtraccion.astimezone(timezone.utc)
#para las consultas a la api solo necesito la fecha, asi que obtengo con el metodo date()
solo_ultima_fecha = ultimaExtraccion.date()


fecha_extraccion = (datetime.now(timezone.utc))
solo_fecha_extraccion = fecha_extraccion.date()



endpoint1='historical-chart'
timeframe = "1hour"
symbol = "ALIENUSD"
#symbol = "BTCUSD"
#from_date= date_now - relativedelta(months=1)

params = {
    "from": solo_ultima_fecha ,
    "to": solo_fecha_extraccion, 
    "apikey": api_key
  
}
endpoint = f"{endpoint1}/{timeframe}/{symbol}"
data = obtener_data(base_url, endpoint, params=params)

if data:
    # Obtener la fecha máxima de los datos recibidos
    fechas_datos = [(d['date']) for d in data if 'date' in d]
    if fechas_datos:
        fecha_maxima_datos = max(fechas_datos)
        print("Fecha máxima:", fecha_maxima_datos)

        # Actualizar la fecha de extracción con la fecha máxima obtenida
        actualizar_fecha_y_proceso(session, fecha_maxima_datos, 'extraccion incremental')
    else:
        print("No se encontraron datos en la respuesta de la API.")
else:
    print("No se obtuvieron datos de la API.")


print(data)

Fecha máxima: 2024-11-18 15:00:00
Fecha y proceso actualizados correctamente.
[{'date': '2024-11-18 15:00:00', 'open': 0.0037169, 'low': 0.0037169, 'high': 0.0037969, 'close': 0.0037969, 'volume': 0.375}, {'date': '2024-11-18 14:00:00', 'open': 0.00379039, 'low': 0.0037169, 'high': 0.00379039, 'close': 0.0037169, 'volume': 0.34375}, {'date': '2024-11-18 13:00:00', 'open': 0.00379039, 'low': 0.00379039, 'high': 0.00379039, 'close': 0.00379039, 'volume': 0.375}, {'date': '2024-11-18 12:00:00', 'open': 0.00378948, 'low': 0.00378948, 'high': 0.00379039, 'close': 0.00379039, 'volume': 0.40625}, {'date': '2024-11-18 11:00:00', 'open': 0.00373434, 'low': 0.00373434, 'high': 0.00378948, 'close': 0.00378948, 'volume': 0.375}, {'date': '2024-11-18 10:00:00', 'open': 0.00367664, 'low': 0.00367664, 'high': 0.00373434, 'close': 0.00373434, 'volume': 0.375}, {'date': '2024-11-18 09:00:00', 'open': 0.00367664, 'low': 0.00367664, 'high': 0.00367664, 'close': 0.00367664, 'volume': 0.375}, {'date': '202

In [ ]:
"""
Llamo a la funcion build_table para convertir el JSON en un DataFrame,
 y coloco un limite de salida para mostrar solo 20 registros del DF
"""
df1= construir_tabla(data,sort_column='date', ascending=True)
pd.set_option('display.max_rows', None) # sirve para mostrar todas las filas completas sin que acortar con ...


df1


,date,open,low,high,close,volume,symbol
17,2024-11-18 00:00:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD
16,2024-11-18 00:32:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD
15,2024-11-18 00:35:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD
14,2024-11-18 01:00:00,0.003712,0.003712,0.003755,0.003755,0.358333,ALIENUSD
13,2024-11-18 02:00:00,0.003755,0.003755,0.003755,0.003755,0.358333,ALIENUSD
12,2024-11-18 03:00:00,0.003755,0.003755,0.003755,0.003755,0.358333,ALIENUSD
11,2024-11-18 04:00:00,0.003755,0.003755,0.003755,0.003755,0.404861,ALIENUSD
10,2024-11-18 05:00:00,0.003755,0.003755,0.003755,0.003755,0.375000,ALIENUSD
9,2024-11-18 06:00:00,0.003755,0.003677,0.003755,0.003677,0.375000,ALIENUSD
8,2024-11-18 07:00:00,0.003677,0.003677,0.003677,0.003677,0.343750,ALIENUSD


Etapa de ingesta en la primera version: Bronze

In [ ]:
"""
En esta parte se realiza una Transformacion de Estandarizacion de fechas, 
donde se convierten las fechas una zona horaria UTC y se localiza a una zona horaria UTC
para los casos donde hay fechas con localizacion none 
"""
path= './delta_table/bronze'

result= guardar_en_tabla_delta(df1,
                        path,
                        partition_cols=None)

No hay datos nuevos para ingresar, la tabla esta actualizada


In [8]:
leer_delta= DeltaTable(path)
dfnuevo= leer_delta.to_pandas()
dfnuevo

,date,open,low,high,close,volume,symbol,__index_level_0__
0,2024-11-18 00:00:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,17
1,2024-11-18 00:32:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,16
2,2024-11-18 00:35:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,15
3,2024-11-18 01:00:00,0.003712,0.003712,0.003755,0.003755,0.358333,ALIENUSD,14
4,2024-11-18 02:00:00,0.003755,0.003755,0.003755,0.003755,0.358333,ALIENUSD,13
5,2024-11-18 03:00:00,0.003755,0.003755,0.003755,0.003755,0.358333,ALIENUSD,12
6,2024-11-18 04:00:00,0.003755,0.003755,0.003755,0.003755,0.404861,ALIENUSD,11
7,2024-11-18 05:00:00,0.003755,0.003755,0.003755,0.003755,0.375000,ALIENUSD,10
8,2024-11-18 06:00:00,0.003755,0.003677,0.003755,0.003677,0.375000,ALIENUSD,9
9,2024-11-18 07:00:00,0.003677,0.003677,0.003677,0.003677,0.343750,ALIENUSD,8


Realizamos algunas modificaciones a las tablas y luego guardamos en la segunda capa: Silver

In [ ]:
# Agregamos una columna id y una columna de symbol que seria el nombre de la cripto


df1['symbol']= symbol
df1.reset_index(drop=True, inplace=True)
df1['id'] = df1.index + 1
df1

,date,open,low,high,close,volume,symbol,id
0,2024-11-18 00:00:00+00:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,1
1,2024-11-18 00:32:00+00:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,2
2,2024-11-18 00:35:00+00:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,3
3,2024-11-18 01:00:00+00:00,0.003712,0.003712,0.003755,0.003755,0.358333,ALIENUSD,4
4,2024-11-18 02:00:00+00:00,0.003755,0.003755,0.003755,0.003755,0.358333,ALIENUSD,5
5,2024-11-18 03:00:00+00:00,0.003755,0.003755,0.003755,0.003755,0.358333,ALIENUSD,6
6,2024-11-18 04:00:00+00:00,0.003755,0.003755,0.003755,0.003755,0.404861,ALIENUSD,7
7,2024-11-18 05:00:00+00:00,0.003755,0.003755,0.003755,0.003755,0.375000,ALIENUSD,8
8,2024-11-18 06:00:00+00:00,0.003755,0.003677,0.003755,0.003677,0.375000,ALIENUSD,9
9,2024-11-18 07:00:00+00:00,0.003677,0.003677,0.003677,0.003677,0.343750,ALIENUSD,10


In [10]:
# convierto la columna symbol a un string que pandas pueda intrepretar como string, porque mas arriba ya lo pase a
#  string pero pandas no reconoce explicitamente entre object y string
df1['symbol'] = df1['symbol'].astype('string')
df1.head(10)


,date,open,low,high,close,volume,symbol,id
0,2024-11-18 00:00:00+00:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,1
1,2024-11-18 00:32:00+00:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,2
2,2024-11-18 00:35:00+00:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,3
3,2024-11-18 01:00:00+00:00,0.003712,0.003712,0.003755,0.003755,0.358333,ALIENUSD,4
4,2024-11-18 02:00:00+00:00,0.003755,0.003755,0.003755,0.003755,0.358333,ALIENUSD,5
5,2024-11-18 03:00:00+00:00,0.003755,0.003755,0.003755,0.003755,0.358333,ALIENUSD,6
6,2024-11-18 04:00:00+00:00,0.003755,0.003755,0.003755,0.003755,0.404861,ALIENUSD,7
7,2024-11-18 05:00:00+00:00,0.003755,0.003755,0.003755,0.003755,0.375000,ALIENUSD,8
8,2024-11-18 06:00:00+00:00,0.003755,0.003677,0.003755,0.003677,0.375000,ALIENUSD,9
9,2024-11-18 07:00:00+00:00,0.003677,0.003677,0.003677,0.003677,0.343750,ALIENUSD,10


In [11]:
# consultar la cantidad de filas
print(f'la cantidad de fila es: {df1.shape[0]}')

la cantidad de fila es: 18


In [12]:
# ingesta silver
path= './delta_table/silver'

result= guardar_en_tabla_delta(df1,
                        path,
                        partition_cols=None)


No hay datos nuevos para ingresar, la tabla esta actualizada


In [13]:
dtable = DeltaTable(path)
d=dtable.to_pandas()
d.head(10)

,date,open,low,high,close,volume,symbol,id
0,2024-11-18 00:00:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,1
1,2024-11-18 00:32:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,2
2,2024-11-18 00:35:00,0.003712,0.003712,0.003712,0.003712,0.000000,ALIENUSD,3
3,2024-11-18 01:00:00,0.003712,0.003712,0.003755,0.003755,0.358333,ALIENUSD,4
4,2024-11-18 02:00:00,0.003755,0.003755,0.003755,0.003755,0.358333,ALIENUSD,5
5,2024-11-18 03:00:00,0.003755,0.003755,0.003755,0.003755,0.358333,ALIENUSD,6
6,2024-11-18 04:00:00,0.003755,0.003755,0.003755,0.003755,0.404861,ALIENUSD,7
7,2024-11-18 05:00:00,0.003755,0.003755,0.003755,0.003755,0.375000,ALIENUSD,8
8,2024-11-18 06:00:00,0.003755,0.003677,0.003755,0.003677,0.375000,ALIENUSD,9
9,2024-11-18 07:00:00,0.003677,0.003677,0.003677,0.003677,0.343750,ALIENUSD,10


In [14]:
print(f'la cantidad de fila es: {d.shape[0]}')

la cantidad de fila es: 18
